<a href="https://colab.research.google.com/github/talalba/Instagram_Like_Predictor/blob/master/lid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the necessary packages
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import importlib
import itertools
import nltk
from nltk import ngrams
from nltk import word_tokenize
import string
import collections
import pandas as pd
import numpy as np
import re
from random import sample 
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from string import digits 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# load the data
directoryPath = '/content/drive/My Drive/lid/'
# df = pd.read_csv(directoryPath+'formatted_data.csv',sep=';',encoding='latin1')
df = pd.read_csv(directoryPath+'formatted_data.csv',sep=';')
df.shape

# get the language data
text = df['text'].tolist()


(21, 3)

In [0]:
# filter the text
def filter_data(text):

  # define digits which will be removed from the string
  remove_digits = str.maketrans('', '', digits) 

  docs = [x.translate(remove_digits) for x in text]   # remove numbers in strings
  docs = [x.lower() for x in docs] # make letters lower case
  docs = [x.strip() for x in docs] # remove white space 
  # docs = [x.translate(str.maketrans('', '', '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~')) for x in docs] # remove punctuation
  # docs = [x.translate(str.maketrans('', '',   string.punctuation )) for x in docs] # remove punctuation

  return docs


In [25]:
filtered_text = filter_data(text)

# get sentences from paragraphs
# ? do periods mark sentences
filtered_text_sen = [sent_tokenize(x) for x in filtered_text]

len(filtered_text_sen[0])
len(text[0])

2744

In [96]:
# get performance stats
no_train_sentences = 200
no_test_sentences = 1
no_runs = 1 # number of times to run the classifier
ngram_number = 1
target_names = df['language'].tolist()
predicted_labels = []
actual_labels = list(range(0,df.shape[0]))*no_runs

# do n numher of runs and get predicted languages 
for ii in tqdm(range(0,no_runs)):
  predicted_labels = np.append(predicted_labels,get_performance_stats(no_train_sentences,no_test_sentences,filtered_text_sen,ngram_number))

# prin classification performance 
cr = classification_report( actual_labels,predicted_labels, target_names=target_names)
print(cr)

# print confusion matrix
cm = confusion_matrix(actual_labels, predicted_labels)
print(cm)

              precision    recall  f1-score   support

          bg       0.50      1.00      0.67         1
          cs       1.00      1.00      1.00         1
          da       1.00      1.00      1.00         1
          de       1.00      1.00      1.00         1
          el       1.00      1.00      1.00         1
          en       1.00      1.00      1.00         1
          es       1.00      1.00      1.00         1
          et       0.00      0.00      0.00         1
          fi       0.50      1.00      0.67         1
          fr       1.00      1.00      1.00         1
          hu       1.00      1.00      1.00         1
          it       1.00      1.00      1.00         1
          lt       1.00      1.00      1.00         1
          lv       0.00      0.00      0.00         1
          nl       1.00      1.00      1.00         1
          pl       1.00      1.00      1.00         1
          pt       1.00      1.00      1.00         1
          ro       1.00    

In [0]:
def get_performance_stats(no_train_sentences,no_test_sentences,data,ngram_number):

  # split_data = lambda x:  sample(x,no_train_sentences + no_test_sentences)

  # z = list(map(split_data, data))

  data = filtered_text_sen

  # split data into train and test set
  z = [sample(x,no_train_sentences + no_test_sentences) for x in data]

  # index the train and test data from the list z
  train_data = [item[:no_train_sentences] for item in z]
  test_data = [item[-no_test_sentences:] for item in z]

  def remove_punctuation(from_text):
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in from_text]
    return stripped

  train_data = [remove_punctuation(i) for i in train_data]
  test_data = [remove_punctuation(i) for i in test_data]

  # combine the list of sentences to a list of strings 
  train_data = [''.join(x) for x in train_data]
  test_data = [''.join(x) for x in test_data]

  # # remove punctuation from the string
  # train_data2 = [remove_punctuation(text) for text in train_data]
  # test_data2 = [remove_punctuation(text) for text in test_data]

  # len(test_data2)

  # pd.DataFrame([D.idxmin(axis=1), pd.Series(target_names)],index=['predicted','actual'])
  # D.idxmin(axis=1).where(D.idxmin(axis=1)==target_names)

  dist_mat = get_language_id(ngram_number,train_data,test_data)

  return dist_mat.idxmin(axis=1).tolist()


  # y_true = df['language'].index.values
  # y_predicted = D.idxmin(axis=1)
  # target_names = df['language'].tolist()
  # # len(target_names)
  # # pd.crosstab(df['language'],D.idxmin(axis=1))

  # print(classification_report( df['language'], D.idxmin(axis=1), target_names=target_names))


In [0]:
# this function performs language identification given train and test data and the ngram number 
# train_data, test_data contain a list for each language

def get_language_id(ngram_number,train_data2,test_data2):
    
  # perform statistics on training data for all languages
  # define function to get unigram frequencies 
  get_frequency_unigram = lambda x:  nltk.FreqDist((ngrams(' '.join(map(str, x)), 2)))
  # get_frequency_unigram = lambda x:  nltk.FreqDist((ngrams(x, ngram_number)))

  # nltk.FreqDist((ngrams(train_data[0], ngram_number)))
  # list((ngrams(' '.join(map(str, train_data[0])), ngram_number)))
  # list(nltk.FreqDist((ngrams(' '.join(map(str, train_data[0])), ngram_number))))                         
  # padded_ngrams = partial(ngrams, pad_left=True, pad_right=True, left_pad_symbol='_', right_pad_symbol='_')

  # ee = [(ngrams(x, 1)) for x in train_data[5]]
  # get ngram counts for train and test data
  # list_ngram_train = [nltk.FreqDist((ngrams(x, ngram_number))) for x in ee]
  list_ngram_train = list(map(get_frequency_unigram, train_data2))

  list_ngram_test = list(map(get_frequency_unigram, test_data2))
  
  # list_unigram_train = list(map(get_frequency_unigram, train_data))
  # list_unigram_test = list(map(get_frequency_unigram, test_data))
  # make dataframe
  train_data_df = pd.DataFrame(list_ngram_train)
  test_data_df = pd.DataFrame(list_ngram_test)

  # test_data_df = pd.DataFrame(list_ngram_test)

  # a count of 0.5 is added to events seen in at least
  # one language in the training set
  np.max(train_data_df.isna().sum())

  # all colums have atleast one nonempty values given how python constructs dictionaries
  # thus we replace all NAs with .5
  train_data_df = train_data_df.fillna(.5)

  # get probabilities from counts by dividing the sum
  train_data_df = train_data_df.div(train_data_df.sum(axis=1), axis=0)
  test_data_df = test_data_df.div(test_data_df.sum(axis=1), axis=0)
  
  try:

    # get common ngram columns between train and test data
    common_cols =  np.intersect1d(train_data_df.columns, test_data_df.columns)
    len(common_cols)

    train_data_df2 = train_data_df[common_cols]
    test_data_df2 = test_data_df[common_cols]

    # D = pd.DataFrame(index=train_data_df.index, columns=train_data_df.index)

    D = pd.DataFrame(columns=train_data_df.index)
    
    # perform language indetification using relative entropy
    for ii in range(0,len(test_data)):

      # compute the KL distance 
      # D = (test_data_df2.iloc[ii,:]*np.log(test_data_df2.iloc[ii,:].div(train_data_df2))).sum(axis=1)
      # D.iloc[ii,:] = (test_data_df2.iloc[ii,:]*np.log(test_data_df2.iloc[ii,:].div(train_data_df2))).sum(axis=1)
      d = (test_data_df2.iloc[ii,:]*np.log(test_data_df2.iloc[ii,:].div(train_data_df2))).sum(axis=1)

      D = D.append(d, ignore_index=True)


      # find index of the train language which has the minimum score (closest language in the train set)
      # index = np.argmin(D.iloc[ii,:])
      
      # print(index)
  except:
    print("No mtaching ngrams found between test and train data")


  return D

In [103]:
# to test one sentence or more
test_sentence = ['I am currently eating my breakfast','I will go to school']
test_sentence = pd.DataFrame(['J’ai oublié mon parapluie dans l’abribus'],columns = ['text'])


filtered_text = filter_data(test_sentence)

# get sentences from paragraphs
# ? do periods mark sentences
filtered_test_sen = [sent_tokenize(x) for x in filtered_text]

# remove punctuation from the string
filtered_test_sen2 = [remove_punctuation(text) for text in filtered_test_sen]

# for unigrams don't include the white space as a feature
if (ngram_number==1):
  test_data2 = [text.replace(" ", "") for text in test_data2] 

len(test_data2)
distance_mat = get_language_id(1,train_data,filtered_test_sen)


# test_data2 = test_data
# train_data2 = train_data2

distance_mat.idxmin(axis=1).to_string(index=False)


IndexError: ignored

In [0]:
# Highlighting the limits of your model

# no spatial order
# size of the feature space increase as bigger n grams are used
# charcter level doesnot capture word level information, combin word and character levle features?
# doesnot work well for bery short sentences as the probailities are not accurate
# use PCA dimentionality reduction
# skipe gram model, graph based tp capture surrounding word distributions 

In [0]:
# # prepare the train and test data

# # split data into train and test set
# test_data_size = .8 # use 20% data for testing, 80% for training

# # get all the words for each language and save it in a list
# word_list = [i.split() for i in  docs]

# # generate training and test data for each language using a map fucntion
# # we will split the words in each language into train and test
# # this ignores the temporal order of the words
# split_data = lambda x:  train_test_split(x,test_size=test_data_size)

# # get a list which contains the test and train data splits
# z = list(map(split_data, word_list))
# # z2 = [train_test_split(x,test_size=.2,radnom_state=42) for x in word_list];

# # index the train and test data from the list z
# train_data = [item[0] for item in z]
# test_data = [item[1] for item in z]